## lxml Implementation

In [1]:
from pathlib import Path
from lxml import etree as et

file_read = Path.cwd() / "files" / "AccountProcess.process"

with open(file_read, "r+") as f:
    tree = et.parse(f)

root = tree.getroot()
root

<Element {http://xmlns.tibco.com/bw/process/2003}ProcessDefinition at 0x106de7900>

### Remove Namespaces using XSLT Transformation

In [2]:
xslt_doc = Path.cwd() / "files" / "xslt_doc.xml"
with open(xslt_doc, "r+") as f:
    xslt = et.parse(f)

transform = et.XSLT(xslt)
result = transform(root)

transformed_doc = Path.cwd() / "files" / "transformed_doc.xml"
result.write(transformed_doc)

### Generate Tibco Mappings as Python Dictionary (k, v) pairs

In [3]:
def get_mapping(mapper):
    mappings = mapper.find("./inputBindings/root/")
    my_dict = {}

    slicer = len(mappings.xpath("ancestor::*"))

    for var in mappings.xpath(".//value-of"):
        # Build the full_path from inputBindings with slicing logic
        ancestors_list = var.xpath("ancestor::*")  # excludes self
        full_path = "/".join(
            ancestor.tag for ancestor in ancestors_list[slicer:] if ancestor.tag not in ("if"))

        my_dict[full_path] = var.attrib["select"].strip('"')
    print(my_dict)

In [4]:
activities = result.findall("./activity")

mapper_list = []
for activity in activities:
    if activity.xpath(".//resourceType='ae.activities.MapperActivity'"):
        mapper_list.append(activity)

if len(mapper_list) == 1:
    get_mapping(mapper_list[0])

{'AcctDetails/RecordIndicator': '$var/RecordType', 'AcctDetails/CustomerID': '$var/CustomerId', 'AcctDetails/AcctNumber': '$var/AccountNumber', 'AcctDetails/TxnID': '$var/TransactionId', 'AcctDetails/TxnTypeCode': '$var/TransactionType', 'AcctDetails/TxnAmount': '$var/TransactionAmount', 'AcctDetails/Currency': '$var/CurrencyCode', 'AcctDetails/TxnDate': '$var/TransactionDate', 'AcctDetails/TxnTime': '$var/TransactionTime', 'AcctDetails/BranchID': '$var/BranchCode', 'AcctDetails/ChannelType': '$var/ChannelCode', 'AcctDetails/StatusCode': '$var/TransactionStatus', 'AcctDetails/CurrentBalance': '$var/AvailableBalance', 'AcctDetails/ProcessDate': '$var/ProcessingDate', 'AcctDetails/AuthCode': 'OAuth', 'AcctDetails/FeeAmount': '0'}
